In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [10]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

### filling Null Values
df = df.fillna(method='ffill')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,35000,0.1308,797.80,MORTGAGE,125890.0,Verified,Mar-2019,Fully Paid,n,30.48,...,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0,N,N
1,20000,0.2250,769.00,RENT,75000.0,Not Verified,Mar-2019,low_risk,n,24.37,...,100.0,100.0,0.0,0.0,77797.0,55014.0,47000.0,30797.0,N,N
2,10500,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
3,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
4,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N


In [11]:
### Getting Dummies Data
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

# Split the Data into Training and Testing

In [12]:
# Create our features
X = df.drop(columns='loan_status')


# # Create our target
yy = {'loan_status': list(df['loan_status'].values)}
y = pd.DataFrame(yy)

In [13]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.0,74.0,74.000000,74.000000,...,74.000000,74.000000,74.000000,74.0,74.000000,74.000000,74.000000,74.000000,74.0,74.0
mean,17952.027027,0.133351,523.842162,1.081081,82199.864865,0.851351,0.0,0.0,18.860946,0.202703,...,95.497297,36.451351,0.148649,0.0,163726.945946,57068.081081,24067.567568,49874.986486,0.0,0.0
std,9715.226678,0.048460,277.888959,0.975828,41472.824072,0.788312,0.0,0.0,11.126093,0.496004,...,8.167711,33.977770,0.358170,0.0,157915.249140,68925.502912,21796.186094,64588.647664,0.0,0.0
min,2400.000000,0.064600,74.730000,0.000000,16323.000000,0.000000,0.0,0.0,0.920000,0.000000,...,54.500000,0.000000,0.000000,0.0,8500.000000,1848.000000,0.000000,0.000000,0.0,0.0
25%,10000.000000,0.083450,318.377500,0.000000,56375.000000,0.000000,0.0,0.0,11.527500,0.000000,...,93.850000,0.000000,0.000000,0.0,51119.000000,18983.500000,9725.000000,12675.000000,0.0,0.0
50%,15900.000000,0.139000,450.385000,2.000000,73750.000000,1.000000,0.0,0.0,17.905000,0.000000,...,100.000000,33.300000,0.000000,0.0,106527.500000,41158.500000,17650.000000,33202.500000,0.0,0.0
75%,25000.000000,0.164000,698.150000,2.000000,100000.000000,1.000000,0.0,0.0,24.042500,0.000000,...,100.000000,64.300000,0.000000,0.0,219247.000000,65435.250000,31325.000000,65339.500000,0.0,0.0
max,40000.000000,0.288000,1537.990000,2.000000,309600.000000,2.000000,0.0,0.0,62.800000,2.000000,...,100.000000,100.000000,1.000000,0.0,588300.000000,418934.000000,106300.000000,383625.000000,0.0,0.0


In [15]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    58
0    16
Name: loan_status, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1,shuffle=True)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [18]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100)
clf.fit(X_train.values, Y_train.values)

BalancedRandomForestClassifier(random_state=1)

In [19]:
pred = clf.predict(X_test.values)

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

balanced_accuracy_score(pred, Y_test.values)

1.0

In [21]:
# Display the confusion matrix
confusion_matrix(pred, Y_test.values)

array([[1, 0],
       [0, 7]])

In [23]:
# Print the imbalanced classification report
import imblearn.metrics as imb
print(imb.classification_report_imbalanced(Y_test.values, pred, target_names=['class_0', 'class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    class_0       1.00      1.00      1.00      1.00      1.00      1.00         1
    class_1       1.00      1.00      1.00      1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8



In [59]:
# List the features sorted in descending order by feature importance
imp = clf.feature_importances_
f_name = columns
sorter = lambda x:x[1]

sorted(zip(f_name, imp), key=sorter, reverse=True)

[('initial_list_status', 0.1575378787878788),
 ('total_pymnt_inv', 0.13960661620084996),
 ('last_pymnt_amnt', 0.12026303130611697),
 ('out_prncp', 0.10261351726576641),
 ('total_pymnt', 0.10062951738019464),
 ('out_prncp_inv', 0.07284754792096991),
 ('collection_recovery_fee', 0.07170411245101649),
 ('pymnt_plan', 0.01909493539548687),
 ('total_rec_prncp', 0.015988399188960334),
 ('open_il_24m', 0.013603152155230882),
 ('num_bc_tl', 0.011768392732587106),
 ('installment', 0.010757257448433918),
 ('mo_sin_old_il_acct', 0.010311360666623826),
 ('open_rv_24m', 0.009710955710955713),
 ('total_bc_limit', 0.009490970177244687),
 ('bc_open_to_buy', 0.007909226190476194),
 ('loan_amnt', 0.007043344273736433),
 ('num_rev_tl_bal_gt_0', 0.006856140350877193),
 ('num_op_rev_tl', 0.006678362573099415),
 ('tot_hi_cred_lim', 0.006488095238095238),
 ('mths_since_rcnt_il', 0.006248106060606061),
 ('total_bal_ex_mort', 0.005878987084344228),
 ('il_util', 0.0058642878503075875),
 ('num_il_tl', 0.00575046

### Easy Ensemble AdaBoost Classifier

In [60]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
clf = EasyEnsembleClassifier(random_state=1, n_estimators=100)
clf.fit(X_train.values, Y_train.values)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [61]:
pred = clf.predict(X_test.values)

In [62]:
# Calculated the balanced accuracy score
balanced_accuracy_score(pred, Y_test.values)

1.0

In [63]:
# Display the confusion matrix
confusion_matrix(pred, Y_test.values)

array([[1, 0],
       [0, 7]])

In [65]:
# Print the imbalanced classification report
print(imb.classification_report_imbalanced(Y_test.values.flatten(), pred,     target_names=['Class_0', 'Class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    Class_0       1.00      1.00      1.00      1.00      1.00      1.00         1
    Class_1       1.00      1.00      1.00      1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8

